<a href="https://colab.research.google.com/github/MSJahnavi/CODSOFT/blob/main/TASK_4_MOVIE_RECOMMENDATION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import warnings; warnings.simplefilter('ignore')

In [2]:
mv=pd.read_csv('/content/movies.csv')
mv.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
mv['genres']=mv['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [4]:
counts=mv[mv['vote_count'].notnull()]['vote_count'].astype('int')
avg=mv[mv['vote_average'].notnull()]['vote_average'].astype('int')
avgprint=avg.mean()
avgprint

5.244896612406511

In [5]:
f=counts.quantile(0.95)
f

434.0

In [6]:
mv['year']=pd.to_datetime(mv['release_date'],errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)



In [7]:
good=mv[(mv['vote_count'] >= f)&(mv['vote_count'].notnull()) & (mv['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
good['vote_count']=good['vote_count'].astype('int')
good['vote_average']=good['vote_average'].astype('int')
good.shape

(2274, 6)

In [8]:
def rating(x):
    v=x['vote_count']
    a=x['vote_average']
    return (v/(v+f)*a)+(f/(f+v)*avgprint)

In [9]:
good['wr']=good.apply(rating, axis=1)

In [10]:
good=good.sort_values('wr', ascending=False).head(250)

In [11]:
good.head()

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787


In [12]:
s=mv.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name='genre'
gen_mv=mv.drop('genres', axis=1).join(s)

In [13]:
def chart(genre,percentile=0.85):
    df=gen_mv[gen_mv['genre'] == genre]
    counts=df[df['vote_count'].notnull()]['vote_count'].astype('int')
    avg=df[df['vote_average'].notnull()]['vote_average'].astype('int')
    avgprint=avg.mean()
    f=counts.quantile(percentile)
    good=df[(df['vote_count']>=f)&(df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    good['vote_count']=good['vote_count'].astype('int')
    good['vote_average']=good['vote_average'].astype('int')
    good['wr'] = good.apply(lambda x: (x['vote_count']/(x['vote_count']+f)*x['vote_average'])+(f/(f+x['vote_count'])*avgprint),axis=1)
    good=good.sort_values('wr',ascending=False).head(250)

    return good


In [14]:
chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


In [15]:
ls=pd.read_csv('/content/movdet.csv')
ls=ls[ls['tmdbId'].notnull()]['tmdbId'].astype('int')

In [16]:
mv=mv.drop([19730, 29503, 35587])

In [17]:
mv['id']=mv['id'].astype('int')

In [18]:
smv=mv[mv['id'].isin(ls)]
smv.shape

(9099, 25)

In [19]:
smv['tagline']=smv['tagline'].fillna('')
smv['description']=smv['overview']+smv['tagline']
smv['description']=smv['description'].fillna('')

In [20]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf=tf.fit_transform(smv['description'])

In [21]:
tfidf.shape

(9099, 268123)

In [22]:
cossim=linear_kernel(tfidf,tfidf)

In [23]:
cossim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [24]:
smv=smv.reset_index()
t=smv['title']
idx=pd.Series(smv.index, index=smv['title'])

In [25]:
def get_recommendations(title):
    ind=idx[title]
    score=list(enumerate(cossim[ind]))
    score=sorted(score,key=lambda x: x[1],reverse=True)
    score=score[1:31]
    movieidx=[i[0] for i in score]
    return t.iloc[movieidx]

In [26]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [27]:
get_recommendations('The Dark Knight').head(20)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
6144                              Batman Begins
7933         Sherlock Holmes: A Game of Shadows
5511                            To End All Wars
4489                                      Q & A
7344                        Law Abiding Citizen
7242                  The File on Thelma Jordon
3537                               Criminal Law
2893                              Flying Tigers
1135                   Night Falls on Manhattan
8680                          The Young Savages
Name: title, dtype: object

In [28]:
get_recommendations('Mean Girls').head(10)

3417           Ice Castles
7118            Wild Child
7495    Death at a Funeral
4990         Doc Hollywood
1084             Cape Fear
1083             Cape Fear
7231            The Clique
4491           Pillow Talk
608              The Craft
5076           Latter Days
Name: title, dtype: object